In [1]:
import pandas as pd

In [2]:
fileurl = "https://github.com/neurone0/explore_data/raw/main/"

In [3]:
filename_train = "melbourne-housing-snapshot/Data/train.csv"
filename_test = "melbourne-housing-snapshot/Data/test.csv"

In [7]:
#Read the data
X_full = pd.read_csv(fileurl+filename_train,index_col="Id")
X_test_full =pd.read_csv(fileurl+filename_test,index_col="Id")

In [8]:
#Obtain target and predictors

In [10]:
X_full.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [11]:
y = X_full.SalePrice

In [13]:
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

In [14]:
X = X_full[features].copy()

In [15]:
X_test = X_test_full[features].copy()

In [16]:
# Break off validation set from training data
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [19]:
X_train.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
619,11694,2007,1828,0,2,3,9
871,6600,1962,894,0,1,2,5
93,13360,1921,964,0,1,2,5
818,13265,2002,1689,0,2,3,7
303,13704,2001,1541,0,2,3,6


In [20]:
#This is testing 5 RF model
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

In [21]:
# To select the best model out of the five, we define a function score_model() below.
# This function returns the mean absolute error (MAE) from the validation set.
# Recall that the best model will obtain the lowest MAE. 

In [22]:
from sklearn.metrics import mean_absolute_error

In [25]:
def score_model(model,X_t = X_train,X_v = X_valid, y_t= y_train,y_v = y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v,preds)

In [26]:
for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

Model 1 MAE: 24015
Model 2 MAE: 23740
Model 3 MAE: 23528
Model 4 MAE: 23996
Model 5 MAE: 23706


In [27]:
scorelist = [score_model(models[i]) for i in range(0,len(models))]
print(scorelist)

[24015.492818003917, 23740.979228636657, 23528.78421232877, 23996.676789668687, 23706.672864217904]


In [30]:
best_model_index = scorelist.index(min(scorelist))

In [31]:
my_model = models[best_model_index]

In [32]:
# Fit the model to the training data
my_model.fit(X, y)

RandomForestRegressor(criterion='absolute_error', random_state=0)

In [33]:
# Generate test predictions
preds_test = my_model.predict(X_test)

In [34]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
# output.to_csv('submission.csv', index=False)

In [36]:
X.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
1,8450,2003,856,854,2,3,8
2,9600,1976,1262,0,2,3,6
3,11250,2001,920,866,2,3,6
4,9550,1915,961,756,1,3,7
5,14260,2000,1145,1053,2,4,9
